#COLAB and Libraries and Paths defined
(uncomment if colab used and change paths accordingly in next section)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os 
import random
import cv2
from tqdm import tqdm
import tensorflow as tf
import pandas as pd

from tensorflow.keras import layers
from tensorflow.keras import preprocessing

from google.colab.patches import cv2_imshow

from numpy import loadtxt
from keras.models import load_model
from sklearn import metrics as skcm

Paths

In [ ]:

densenet_path = "/content/drive/MyDrive/FromINKA/Models/densenet_holdout_single_30eps_2.h5"
resnet_path = "/content/drive/MyDrive/FromINKA/Models/resnet50v2_holdout_single_50eps.h5"
efficientnet_path = "/content/drive/MyDrive/FromINKA/Models/efficientnet_holdout_single_30eps_2.h5"

datadir = "/content/drive/MyDrive/data/comb_dt_splits/prediction/"

#Trained Model Load and Data Load into Numpy array with respective labels

In [ ]:
# load all three trained models

densenet_trained_model = load_model(densenet_path)
resnet_trained_model = load_model(resnet_path)
efficientnet_trained_model = load_model(efficientnet_path)


##  Summaries respectively
densenet_trained_model.summary()
resnet_trained_model.summary()
efficientnet_trained_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 densenet121 (Functional)    (None, 2)                 7039554   
                                                                 
Total params: 7,039,554
Trainable params: 6,955,906
Non-trainable params: 83,648
_________________________________________________________________
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50v2 (Functional)     (None, 2)                 23568898  
                                                                 
Total params: 23,568,898
Traina

In [ ]:
DATADIR = datadir # Prediction directory
FOLDERS = ["B","M"]
CATEGORIES = ["B", "M"] 
NUM_CLASSES = 2
IMG_SIZE = 224
class_names = ["Benign", "Malignant"] 

Data read function

In [ ]:
def read_data():
    images = [] 
    labels = []
    img_paths = []
    data = []

    for category,folder in zip(CATEGORIES,FOLDERS):     # do B and M 
        path = os.path.join(DATADIR,folder)             # create path to B and M  
        class_num = CATEGORIES.index(category)          # get the classification  (0 or a 1). 0=B and 1=M
        
        for i,img_path in enumerate(tqdm(os.listdir(path))):              # iterate over each image 
            try:
                img_array = cv2.imread(os.path.join(path,img_path))       # convert to array
                img_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))   # resize to normalize data size                
                img_paths.append(os.path.join(path,img_path))
                
                data.append([img_array, class_num])   # add this to our data
                images.append(img_array)
                labels.append(class_num)
            except Exception as e:                    # in the interest in keeping the output clean...
                pass

    images = np.array(images)                         # output images into np array
    images = np.asarray(images).astype(np.float32)
    labels = np.array(labels)                         # output images into np array
    

    return images,labels

Data read function call and length check

In [ ]:
images,labels = read_data()      # Function call read data
print(len(images))               # to check the length
print(len(labels))
images = images/255 #USes computation power

100%|██████████| 686/686 [00:03<00:00, 211.65it/s]


#Predict and Create Ensemble model

Define predcition function

In [ ]:
def get_predict_score(current_image, selectedModel):
  predictions = 0
  predictions = selectedModel.predict(current_image) #predicts the load image in focus
  return predictions.argmax()                        #outputs the result converting into binary classification

In [ ]:
densenet_predicted = []
resnet_predicted = []
efficientnet_predicted = []

Load unseen Data and predict their labels

In [ ]:

for img in images:
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.expand_dims(img, 0)

  densenet_predicted.append(  get_predict_score(img , densenet_trained_model))
  resnet_predicted.append( get_predict_score(img , resnet_trained_model))
  efficientnet_predicted.append(   get_predict_score(img , efficientnet_trained_model))


Create Averaging Ensemble

In [ ]:
ensemble = []

##Total images being 2222

for i in range (0,2221):

  ##Averaging ensemble calculated and rounded off to convert to binary classification
  ens = round((densenet_predicted[i] + resnet_predicted[i] + efficientnet_predicted[i])/3)
  
  ensemble.append(ens)

In [ ]:
print (ensemble)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

#Confusion matrix and Metrics

Confusion matrix call

In [ ]:
skcm.confusion_matrix(labels, ensemble)


array([[1461,   74],
       [  84,  602]])

Extracting individual parameters from confusion matrix

In [ ]:
tn, fp, fn, tp = skcm.confusion_matrix(labels, ensemble).ravel()

Calculating metrics for the Ensemble model

In [ ]:

Accuracy= (tp+tn)/(fp+tp+tn+fn)
print ("Accuracy:  " +str(Accuracy) )

Specificity= tn/(tn+fp)
print ("Specificity:  " +str(Specificity) )

Sensitivity= (tp)/(tp+fn)
print ("Sensitivity:  " +str(Sensitivity) )


Accuracy:  0.9288608734804142
Specificity:  0.9517915309446254
Sensitivity:  0.8775510204081632
